# Petrophysical Modeling with Amazon SageMaker XGBoost algorithm
_**Single machine training for regression with Amazon SageMaker XGBoost algorithm**_

---

---
## Contents
1. [Introduction](#Introduction)
2. [Setup](#Setup)
  1. [Fetching the dataset](#Fetching-the-dataset)
  2. [Data Ingestion](#Data-ingestion)
3. [Training the XGBoost model](#Training-the-XGBoost-model)
4. [Set up hosting for the model](#Set-up-hosting-for-the-model)
  1. [Import model into hosting](#Import-model-into-hosting)
  2. [Create endpoint configuration](#Create-endpoint-configuration)
  3. [Create endpoint](#Create-endpoint)
5. [Validate the model for use](#Validate-the-model-for-use)

---
## Introduction

This notebook demonstrates the use of Amazon SageMaker's implementation of the XGBoost algorithm to train and host a regression model for prediction of porosity from other well log measurements. 

> A Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text. - http://jupyter.org/

The dataset is log data from nine wells, with 5 wireline log measurements, two indicator variables and a facies label at half foot intervals. The seven predictor variables are:

Five wireline log curves including: 

- Gamma ray (GR)
- Resistivity logging (ILD_log10)
- Photoelectric effect (PE) - Note, some wells do not have PE.
- Neutron-density porosity difference 
- Average neutron-density porosity (DeltaPHI and PHIND)

Two geologic constraining variables: 

- Nonmarine-marine indicator (NM_M) 
- Relative position (RELPOS)

We will focus on these logs: GR, ILD_log10, DeltaPH, PHIND, where we try to predict PHIND by training using 7 wells, 1 validation well, and one well for blind test. 

The dataset comes from a class exercise from The University of Kansas on Neural Networks and Fuzzy Systems. This exercise is based on a consortium project to use machine learning techniques to create a reservoir model of the largest gas fields in North America, the Hugoton and Panoma Fields. For more info on the origin of the data, see Bohling and Dubois (2003) and Dubois et al. (2007).

---
## Setup


This notebook was created and tested on an ml.m4.4xlarge notebook instance.

Let's start by specifying:
1. The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
1. The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
%%time

import os
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()
region = boto3.Session().region_name

# bucket='mef-test' # put your s3 bucket name here, and create s3 bucket
# # prefix = 'mehdi-test/high-level'
# prefix = 'public-data-example/high-level'


bucket='sdu-sagemaker-workshop' # put your s3 bucket name here, and create s3 bucket
prefix = 'public-data-example/high-level'

bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

### Fetching the dataset

Following methods split the data into train/test/validation datasets and upload files to S3.

In [ ]:
%%time

import io
import boto3
import random

def data_split(FILE_DATA, FILE_TRAIN, FILE_VALIDATION, FILE_TEST, Valid_Well_Name, Blind_Well_Name, TARGET_VAR):
    data = pd.read_csv(FILE_DATA)
    n = data.shape[0]
    
    # Make the first column the target feature    
    cols = data.columns.tolist()
    target_pos = data.columns.get_loc(TARGET_VAR)
    cols.pop(target_pos)
    cols = [TARGET_VAR] + cols
    data = data.loc[:,cols]
                
    # Remove target colun from test set
    #test_data = test_data.drop([TARGET_VAR], axis=1)

    # Split data
    test_data = data[data['Well Name'] == Blind_Well_Name]
    test_data = test_data.drop(['Well Name'], axis=1)
    
    valid_data = data[data['Well Name'] == Valid_Well_Name]
    valid_data = valid_data.drop(['Well Name'], axis=1)

    train_data = data[data['Well Name'] != Blind_Well_Name]
    train_data = train_data[train_data['Well Name'] != Valid_Well_Name]

    train_data = train_data.drop(['Well Name'], axis=1)

    train_data.to_csv(FILE_TRAIN, index=False, header=False)
    valid_data.to_csv(FILE_VALIDATION, index=False, header=False)
    test_data.to_csv(FILE_TEST, index=False, header=False)
    
def write_to_s3(fobj, bucket, key):
    return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(fobj)

# def upload_to_s3(bucket, channel, filename):
#     fobj=open(filename, 'rb')
#     key = prefix+'/'+channel+'/'+filename
#     url = 's3://{}/{}'.format(bucket, key)
#     print('Writing to {}'.format(url))
#     write_to_s3(fobj, bucket, key)
#     return(url)


def upload_to_s3(bucket, channel, filename):
    fobj=open(filename, 'rb')
    key = prefix+'/'+channel+'/'+filename
    url = 's3://{}/{}'.format(bucket, key)
    print('Writing to {}'.format(url))
    write_to_s3(fobj, bucket, key)     
    return(url)


### Data ingestion

Next, we read the dataset from the existing repository into memory, for preprocessing prior to training. This processing could be done *in situ* by Amazon Athena, Apache Spark in Amazon EMR, Amazon Redshift, etc., assuming the dataset is present in the appropriate location. Then, the next step would be to transfer the data to S3 for use in training. For small datasets, such as this one, reading into memory isn't onerous, though it would be for larger datasets.

When using the csv option, here is the critical piece of information about the data format:

> For CSV training, the algorithm assumes that the target variable is in the first column and that the CSV does not have a header record. For CSV inference, the algorithm assumes that CSV input does not have the label column.

[Source](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)

In [ ]:
# Remove non-numeric columns
import pandas as pd
data = pd.read_csv('facies_vectors.csv')
# Remove rows with missing values
data.dropna(inplace=True)

data['Well Name'].unique()
data = data.loc[:,['Well Name', 'GR', 'ILD_log10', 'DeltaPHI', 'PHIND']]


# Write file back to disk
data.to_csv('facies_num.csv', index=False)

In [ ]:
FILE_DATA = 'facies_num.csv'
TARGET_VAR = 'PHIND'
FILE_TRAIN = 'facies_train.csv'
FILE_VALIDATION = 'facies_validation.csv'
FILE_TEST = 'facies_test.csv'
Valid_Well_Name = 'SHRIMPLIN'
Blind_Well_Name = 'SHANKLE'

data_split(FILE_DATA, FILE_TRAIN, FILE_VALIDATION, FILE_TEST, Valid_Well_Name, Blind_Well_Name, TARGET_VAR)

In [ ]:
# upload the files to the S3 bucket
s3_train_loc = upload_to_s3(bucket = bucket, channel = 'train', filename = FILE_TRAIN)
s3_valid_loc = upload_to_s3(bucket = bucket, channel = 'validation', filename = FILE_VALIDATION)
s3_test_loc = upload_to_s3(bucket = bucket, channel = 'test', filename = FILE_TEST)

## Training the XGBoost model

After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes between 5 and 6 minutes.

XGBoost hyperparameter [docs](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html).

In [ ]:
import sagemaker

# Instantiate the estimator
xgboost = sagemaker.estimator.Estimator('811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path='s3://{}/{}/output'.format(bucket, prefix),
                                       sagemaker_session=sagemaker.Session())

# Set the hyperparameters
xgboost.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='reg:linear',
                        num_round=100)

# Set the input data formatting and locatiopns
s3_input_train = sagemaker.s3_input(s3_data=s3_train_loc, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=s3_valid_loc, content_type='csv')

# Train the model
xgboost.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
xgboost_predictor = xgboost.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Validate the model for use
Finally, the customer can now validate the model for use. They can obtain the endpoint from the client library using the result from previous operations, and generate classifications from the trained model using that endpoint.


In [ ]:
from sagemaker.predictor import csv_serializer

## import numpy as np

def predict(data, rows=500):
    xgboost_predictor.content_type = 'text/csv'
    xgboost_predictor.serializer = csv_serializer
    xgboost_predictor.deserializer = None

    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgboost_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

test_data = pd.read_csv(FILE_TEST, header=None)
labels = test_data.iloc[:,0]
predictions = predict(test_data.as_matrix()[:, 1:]) # Note that we are not using the first column, which is the label

In [ ]:
import numpy as np

# def predict(data, rows=500):
def predict(data):
    rows=len(data)
    xgboost_predictor.content_type = 'text/csv'
    xgboost_predictor.serializer = csv_serializer
    xgboost_predictor.deserializer = None

    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgboost_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

test_data = pd.read_csv(FILE_TEST, header=None)
labels = test_data.iloc[:,0]
predictions = predict(test_data.as_matrix()[:, 1:]) # Note that we are not using the first column, which is the label

In [ ]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

def log_plot(logs):
    #make sure logs are sorted by depth
    logs = logs.sort_values(by='Depth')

    ztop=logs.Depth.min(); zbot=logs.Depth.max()
        
    f, ax = plt.subplots(nrows=1, ncols=4, figsize=(8, 12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.ILD_log10, logs.Depth, '-')
    ax[2].plot(logs.DeltaPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.PHIND, logs.Depth, '-', color='r')
    ax[3].plot(logs.PPoro, logs.Depth, '-', color='blue')
    ax[3].legend(['Measured','Predicted'])
    
    for i in range(len(ax)):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)
    
    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("ILD_log10")
    ax[1].set_xlim(logs.ILD_log10.min(),logs.ILD_log10.max())
    ax[2].set_xlabel("DeltaPHI")
    ax[2].set_xlim(logs.DeltaPHI.min(),logs.DeltaPHI.max())
    ax[3].set_xlabel("Porosity")
    ax[3].set_xlim(logs.PHIND.min(),logs.PHIND.max())

    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well Name'], fontsize=14,y=0.94)


    
data = pd.read_csv('facies_vectors.csv')
Blindwell = data[data['Well Name'] == Blind_Well_Name]
Blindwell['PPoro'] = predictions
display(Blindwell.head())
log_plot(Blindwell)
print('Correlation coeficient = {0:.5f} \nMean Squared Error = {1:.5f}'.format(
    np.corrcoef(predictions,labels)[0,1]
    , mean_squared_error(predictions,labels)))

### (Optional) Delete the Endpoint
If you're ready to be done with this notebook, make sure run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
print(xgboost_predictor.endpoint)

import sagemaker
sagemaker.Session().delete_endpoint(xgboost_predictor.endpoint)